## Read Data

In [1]:
import pandas as pd
data = pd.DataFrame.from_csv("/host/Users/Simo/Desktop/equota/hvac load prediction/20170315cooling_load/code/out.csv", sep='\t', encoding='utf-8')

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:2: FutureWarning: from_csv is deprecated. Please use read_csv(...) instead. Note that some of the default arguments are different, so please refer to the documentation for from_csv when changing your function calls
  


## Split into Train and Test

In [2]:
from sklearn import cross_validation
X_train, X_test, y_train, y_test=\
cross_validation.train_test_split(data.drop(['cooling_load'],axis = 1), data['cooling_load'],\
                                  test_size=0.3, random_state=0)

/usr/local/lib/python2.7/dist-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


## Data Preprocessing

In [5]:
import numpy as np
Train_Matrix = X_train.as_matrix()
Test_Matrix = X_test.as_matrix()

temp1 = []
for i in Train_Matrix:
    
    temp1.append(np.array([i]))
    
Train_Matrix = np.array(temp1,dtype=np.float32)

temp1 = []
for i in Test_Matrix:
    
    temp1.append(np.array([i]))
    #temp1.append(np.array(temp))
    
Test_Matrix = np.array(temp1,dtype=np.float32)


temp = []
for i in y_train:
    temp.append([i])
y_train = np.asmatrix(temp,dtype=np.float32)

temp = []
for i in y_test:
    temp.append([i])
y_test = np.asmatrix(temp,dtype=np.float32)



## Network generation functions

In [8]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.python.framework import dtypes
from tensorflow.contrib import learn as tflearn
from tensorflow.contrib import layers as tflayers
from tensorflow.contrib import rnn
import warnings

warnings.filterwarnings("ignore")

def rnn_data(data, time_steps, labels=False):
    """
    creates new data frame based on previous observation
      * example:
        l = [1, 2, 3, 4, 5]
        time_steps = 2
        -> labels == False [[1, 2], [2, 3], [3, 4]] #Data frame for input with 2 timesteps
        -> labels == True [3, 4, 5] # labels for predicting the next timestep
    """
    rnn_df = []
    for i in range(len(data) - time_steps):
        if labels:
            try:
                rnn_df.append(data.iloc[i + time_steps].as_matrix())
            except AttributeError:
                rnn_df.append(data.iloc[i + time_steps])
        else:
            data_ = data.iloc[i: i + time_steps].as_matrix()
            rnn_df.append(data_ if len(data_.shape) > 1 else [[i] for i in data_])

    return np.array(rnn_df, dtype=np.float32)



def split_data(data, val_size=0.1, test_size=0.1):
    """
    splits data to training, validation and testing parts
    """
    ntest = int(round(len(data) * (1 - test_size)))
    nval = int(round(len(data.iloc[:ntest]) * (1 - val_size)))

    df_train, df_val, df_test = data.iloc[:nval], data.iloc[nval:ntest], data.iloc[ntest:]

    return df_train, df_val, df_test



def prepare_data(data, time_steps, labels=False, val_size=0.1, test_size=0.1):
    """
    Given the number of `time_steps` and some data,
    prepares training, validation and test data for an lstm cell.
    """
    df_train, df_val, df_test = split_data(data, val_size, test_size)
    return (rnn_data(df_train, time_steps, labels=labels),
            rnn_data(df_val, time_steps, labels=labels),
            rnn_data(df_test, time_steps, labels=labels))

def load_csvdata(rawdata, time_steps, seperate=False):
    data = rawdata
    if not isinstance(data, pd.DataFrame):
        data = pd.DataFrame(data)

    train_x, val_x, test_x = prepare_data(data['a'] if seperate else data, time_steps)
    train_y, val_y, test_y = prepare_data(data['b'] if seperate else data, time_steps, labels=True)
    return dict(train=train_x, val=val_x, test=test_x), dict(train=train_y, val=val_y, test=test_y)


def generate_data(fct, x, time_steps, seperate=False):
    """generates data with based on a function fct"""
    data = fct(x)
    if not isinstance(data, pd.DataFrame):
        data = pd.DataFrame(data)
    train_x, val_x, test_x = prepare_data(data['a'] if seperate else data, time_steps)
    train_y, val_y, test_y = prepare_data(data['b'] if seperate else data, time_steps, labels=True)
    return dict(train=train_x, val=val_x, test=test_x), dict(train=train_y, val=val_y, test=test_y)


def lstm_model(time_steps, rnn_layers, dense_layers=None, learning_rate=0.01, optimizer='Adagrad',learning_rate_decay_fn = None): # [Ftrl, Adam, Adagrad, Momentum, SGD, RMSProp]
    print(time_steps)
    #exit(0)
    """
        Creates a deep model based on:
            * stacked lstm cells
            * an optional dense layers
        :param num_units: the size of the cells.
        :param rnn_layers: list of int or dict
                             * list of int: the steps used to instantiate the `BasicLSTMCell` cell
                             * list of dict: [{steps: int, keep_prob: int}, ...]
        :param dense_layers: list of nodes for each layer
        :return: the model definition
        """

    def lstm_cells(layers):
        print('-------------------------sdsdsdsdssd---------------------------------------------',layers)
        if isinstance(layers[0], dict):
            return [rnn.DropoutWrapper(rnn.BasicLSTMCell(layer['num_units'],state_is_tuple=True),layer['keep_prob'])
                    if layer.get('keep_prob')
                    else rnn.BasicLSTMCell(layer['num_units'], state_is_tuple=True)
                    for layer in layers]

        return [rnn.BasicLSTMCell(steps, state_is_tuple=True) for steps in layers]

    def dnn_layers(input_layers, layers):
        if layers and isinstance(layers, dict):
            return tflayers.stack(input_layers, tflayers.fully_connected,
                                  layers['layers'],
                                  activation=layers.get('activation'),
                                  dropout=layers.get('dropout'))
        elif layers:
            return tflayers.stack(input_layers, tflayers.fully_connected, layers)
        else:
            return input_layers

    def _lstm_model(X, y):
        stacked_lstm = rnn.MultiRNNCell(lstm_cells(rnn_layers), state_is_tuple=True)
        x_ =  tf.unstack(X, num=time_steps, axis=1)


        output, layers = rnn.static_rnn(stacked_lstm, x_, dtype=dtypes.float32)
        output = dnn_layers(output[-1], dense_layers)
        prediction, loss = tflearn.models.linear_regression(output, y)
        train_op = tf.contrib.layers.optimize_loss(
            loss, tf.contrib.framework.get_global_step(), optimizer=optimizer,
            learning_rate = tf.train.exponential_decay(learning_rate, tf.contrib.framework.get_global_step(), decay_steps = 1000, decay_rate = 0.9, staircase=False, name=None))

        print('learning_rate',learning_rate)
        return prediction, loss, train_op

    # https://www.tensorflow.org/versions/r0.10/api_docs/python/train/decaying_the_learning_rate

    return _lstm_model

/usr/local/lib/python2.7/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


## Train and Test

In [25]:
import numpy as np
import pandas as pd
import tensorflow as tf
from matplotlib import pyplot as plt
import warnings

from tensorflow.contrib import learn
from tensorflow.contrib.learn.python import SKCompat
from sklearn.metrics import mean_squared_error

LOG_DIR = 'resources/logs/'
TIMESTEPS = 1
RNN_LAYERS = [300,100]#[{'num_units': 300}]
DENSE_LAYERS = [80,40,20]
TRAINING_STEPS = 50000
PRINT_STEPS = TRAINING_STEPS # / 10
BATCH_SIZE = 100

regressor = SKCompat(learn.Estimator(model_fn=lstm_model(TIMESTEPS, RNN_LAYERS, DENSE_LAYERS),))
                          #   model_dir=LOG_DIR)

validation_monitor = learn.monitors.ValidationMonitor(Test_Matrix, y_test,)
    
    
print('X train shape', Train_Matrix.shape)
print('y train shape', y_train.shape)


    
print('X test shape', Test_Matrix.shape)
print('y test shape', y_test.shape)


SKCompat(regressor.fit(Train_Matrix, y_train,
              monitors=[validation_monitor],
              batch_size=BATCH_SIZE,
              steps=TRAINING_STEPS))


predicted = np.asmatrix(regressor.predict(Test_Matrix),dtype = np.float32) #,as_iterable=False))
predicted = np.transpose(predicted)

rmse = np.sqrt((np.asarray((np.subtract(predicted, y_test))) ** 2).mean()) 
score = mean_squared_error(predicted, y_test)
nmse = score / np.var(y_test) # should be variance of original data and not data from fitted model, worth to double check

print("RSME: %f" % rmse)
print("NSME: %f" % nmse)
print("MSE: %f" % score)

1
INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_save_checkpoints_secs': 600, '_num_ps_replicas': 0, '_keep_checkpoint_max': 5, '_task_type': None, '_is_chief': True, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fb790d556d0>, '_model_dir': '/tmp/tmpTikfh4', '_save_checkpoints_steps': None, '_keep_checkpoint_every_n_hours': 10000, '_session_config': None, '_tf_random_seed': None, '_save_summary_steps': 100, '_environment': 'local', '_num_worker_replicas': 0, '_task_id': 0, '_log_step_count_steps': 100, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1.0
}
, '_evaluation_master': '', '_master': ''}
('X train shape', (2411, 1, 37))
('y train shape', (2411, 1))
('X test shape', (1034, 1, 37))
('y test shape', (1034, 1))
('-------------------------sdsdsdsdssd---------------------------------------------', [100, 100])
('learning_rate', 0.01)
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finaliz

INFO:tensorflow:loss = 23563938.0, step = 6401 (0.494 sec)
INFO:tensorflow:global_step/sec: 166.881
INFO:tensorflow:loss = 22436554.0, step = 6501 (0.601 sec)
INFO:tensorflow:global_step/sec: 193.316
INFO:tensorflow:loss = 20583386.0, step = 6601 (0.515 sec)
INFO:tensorflow:global_step/sec: 168.119
INFO:tensorflow:loss = 25528292.0, step = 6701 (0.596 sec)
INFO:tensorflow:global_step/sec: 167.442
INFO:tensorflow:loss = 20451490.0, step = 6801 (0.599 sec)
INFO:tensorflow:global_step/sec: 177.3
INFO:tensorflow:loss = 28388826.0, step = 6901 (0.563 sec)
INFO:tensorflow:global_step/sec: 197.749
INFO:tensorflow:loss = 17633190.0, step = 7001 (0.508 sec)
INFO:tensorflow:global_step/sec: 203.158
INFO:tensorflow:loss = 16381057.0, step = 7101 (0.489 sec)
INFO:tensorflow:global_step/sec: 184.257
INFO:tensorflow:loss = 17295482.0, step = 7201 (0.543 sec)
INFO:tensorflow:global_step/sec: 217.327
INFO:tensorflow:loss = 23780252.0, step = 7301 (0.461 sec)
INFO:tensorflow:global_step/sec: 236.569
IN

INFO:tensorflow:global_step/sec: 163.853
INFO:tensorflow:loss = 16846292.0, step = 14601 (0.612 sec)
INFO:tensorflow:global_step/sec: 157.679
INFO:tensorflow:loss = 17064960.0, step = 14701 (0.647 sec)
INFO:tensorflow:global_step/sec: 112.286
INFO:tensorflow:loss = 18124574.0, step = 14801 (0.896 sec)
INFO:tensorflow:global_step/sec: 109.881
INFO:tensorflow:loss = 18764866.0, step = 14901 (0.898 sec)
INFO:tensorflow:global_step/sec: 142.726
INFO:tensorflow:loss = 21543942.0, step = 15001 (0.708 sec)
INFO:tensorflow:global_step/sec: 139.119
INFO:tensorflow:loss = 23417564.0, step = 15101 (0.711 sec)
INFO:tensorflow:global_step/sec: 119.821
INFO:tensorflow:loss = 23163870.0, step = 15201 (0.839 sec)
INFO:tensorflow:global_step/sec: 154.187
INFO:tensorflow:loss = 18977594.0, step = 15301 (0.641 sec)
INFO:tensorflow:global_step/sec: 178.87
INFO:tensorflow:loss = 16883426.0, step = 15401 (0.553 sec)
INFO:tensorflow:global_step/sec: 166.004
INFO:tensorflow:loss = 17626832.0, step = 15501 (0.

INFO:tensorflow:loss = 16760855.0, step = 22701 (0.506 sec)
INFO:tensorflow:global_step/sec: 204.733
INFO:tensorflow:loss = 12889559.0, step = 22801 (0.488 sec)
INFO:tensorflow:global_step/sec: 188.748
INFO:tensorflow:loss = 18943464.0, step = 22901 (0.531 sec)
INFO:tensorflow:global_step/sec: 167.909
INFO:tensorflow:loss = 19664314.0, step = 23001 (0.594 sec)
INFO:tensorflow:global_step/sec: 168.222
INFO:tensorflow:loss = 23996516.0, step = 23101 (0.597 sec)
INFO:tensorflow:global_step/sec: 171.571
INFO:tensorflow:loss = 26284132.0, step = 23201 (0.581 sec)
INFO:tensorflow:global_step/sec: 179.193
INFO:tensorflow:loss = 20353472.0, step = 23301 (0.558 sec)
INFO:tensorflow:global_step/sec: 190.413
INFO:tensorflow:loss = 17747788.0, step = 23401 (0.530 sec)
INFO:tensorflow:global_step/sec: 221.909
INFO:tensorflow:loss = 16915660.0, step = 23501 (0.444 sec)
INFO:tensorflow:global_step/sec: 208.924
INFO:tensorflow:loss = 10636152.0, step = 23601 (0.480 sec)
INFO:tensorflow:global_step/sec

INFO:tensorflow:global_step/sec: 174.509
INFO:tensorflow:loss = 16126785.0, step = 30901 (0.572 sec)
INFO:tensorflow:global_step/sec: 178.326
INFO:tensorflow:loss = 16658871.0, step = 31001 (0.563 sec)
INFO:tensorflow:global_step/sec: 179.346
INFO:tensorflow:loss = 16290902.0, step = 31101 (0.555 sec)
INFO:tensorflow:global_step/sec: 164.235
INFO:tensorflow:loss = 20794994.0, step = 31201 (0.610 sec)
INFO:tensorflow:global_step/sec: 176.417
INFO:tensorflow:loss = 21147080.0, step = 31301 (0.570 sec)
INFO:tensorflow:global_step/sec: 163.865
INFO:tensorflow:loss = 23213282.0, step = 31401 (0.608 sec)
INFO:tensorflow:global_step/sec: 191.839
INFO:tensorflow:loss = 27076770.0, step = 31501 (0.520 sec)
INFO:tensorflow:global_step/sec: 197.066
INFO:tensorflow:loss = 18555522.0, step = 31601 (0.508 sec)
INFO:tensorflow:global_step/sec: 169.404
INFO:tensorflow:loss = 17528358.0, step = 31701 (0.591 sec)
INFO:tensorflow:global_step/sec: 198.861
INFO:tensorflow:loss = 15672096.0, step = 31801 (0

INFO:tensorflow:loss = 16054692.0, step = 39001 (0.493 sec)
INFO:tensorflow:global_step/sec: 216.915
INFO:tensorflow:loss = 13689478.0, step = 39101 (0.461 sec)
INFO:tensorflow:global_step/sec: 200.19
INFO:tensorflow:loss = 14908616.0, step = 39201 (0.502 sec)
INFO:tensorflow:global_step/sec: 193.338
INFO:tensorflow:loss = 18728418.0, step = 39301 (0.515 sec)
INFO:tensorflow:global_step/sec: 171.33
INFO:tensorflow:loss = 13535822.0, step = 39401 (0.586 sec)
INFO:tensorflow:global_step/sec: 216.197
INFO:tensorflow:loss = 18432526.0, step = 39501 (0.463 sec)
INFO:tensorflow:global_step/sec: 215.047
INFO:tensorflow:loss = 17509192.0, step = 39601 (0.462 sec)
INFO:tensorflow:global_step/sec: 235.374
INFO:tensorflow:loss = 20865064.0, step = 39701 (0.424 sec)
INFO:tensorflow:global_step/sec: 214.561
INFO:tensorflow:loss = 16485930.0, step = 39801 (0.465 sec)
INFO:tensorflow:global_step/sec: 225.843
INFO:tensorflow:loss = 13017554.0, step = 39901 (0.444 sec)
INFO:tensorflow:global_step/sec: 

KeyboardInterrupt: 

## Multi time steps